In [ ]:
from skimage import io

from skimage.filters import roberts, scharr, prewitt, sobel, hessian, frangi, sato, threshold_yen
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.exposure import equalize_hist
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import erosion, dilation
from skimage.feature import greycomatrix, greycoprops

from matplotlib import pyplot as plt

import os
import numpy as np
import pandas as pd

from skimage.filters import gaussian

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

from matplotlib import pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np
from skimage import io, color, img_as_ubyte
from skimage.measure import label, regionprops, regionprops_table
from skimage.feature import hog, corner_moravec, greycomatrix, greycoprops, canny
from skimage.exposure import equalize_hist
from skimage.filters import sato
from skimage import data, exposure, feature
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix 
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


import os
import glob


from scipy.stats import mannwhitneyu, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd


"""
HOG + HARALICK
"""


In [ ]:
def HOGex(img, path):
    H = feature.hog(img, orientations=orientation, pixels_per_cell=pixelspercell, cells_per_block=cellsperblock,
            transform_sqrt=True, block_norm="L1")
    return H
def GLCMex(img, path):
    glcm = greycomatrix(img, 
            distances=distances, 
            angles=angles,
            symmetric=symmetric,
            normed=normed)
    properties=['contrast', 'dissimilarity', 'homogeneity', 'energy',
                        'correlation', 'ASM']
    feats = np.hstack([greycoprops(glcm, prop).ravel() for prop in properties])
    return feats



def imageloader(paths, filter_on=False, norm=False, test=False, properties=['contrast', 'dissimilarity', 'homogeneity', 'energy',
                          'correlation', 'ASM']):
    for path in paths:
        print(path)
        for i, image_path in enumerate(os.listdir(path)):
            if not image_path.startswith('.'):
                if test == True:
                    if i > 5:
                        break
                print(f"{i+1} of {len(os.listdir(path))}")  
                input_path = os.path.join(path, image_path)
                img = io.imread(input_path, as_gray=True)
                img = resize(img, (yy, xx))
                if norm == True:
                    img= equalize_hist(img)
                if filter_on == True:
                    img= sato(img)
                img = img_as_ubyte(img)
                """
                featureextraction
                """
                H = HOGex(img, path)
                feats = GLCMex(img, path)
                """
                store
                """
                datahog.append(H)
                dataglcm.append(feats)
                globalfeat = np.hstack([H, feats])
                globalfeats.append(globalfeat)
                if path == paths[0]:
                    flg=1
                    globallabels.append(flg)
                    labelsglcm.append(flg)
                    labelshog.append(flg)
                if path == paths[1]:
                    flg=0
                    globallabels.append(flg)
                    labelsglcm.append(flg)
                    labelshog.append(flg)

def GLCMHOGtest():
    for name, model in models:
        kfold = KFold(n_splits=kfoldsplits) #random_state=seed)
        cv_results = cross_val_score(model, trainDataGlobal, trainLabelsGlobal, cv=kfold, scoring=scoring)
        results.append(cv_results)
        print(f"results: {cv_results}")
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    # boxplot algorithm comparison
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.boxplot(results)
#    showmeans=True
    ax.set_xticklabels(names)
    plt.show()
    
def GLCMHOGtest2():
    for name, model in models:
        kfold = KFold(n_splits=kfoldsplits) #random_state=seed)
        cv_results = cross_val_score(model, trainDataGlobal, trainLabelsGlobal, cv=kfold, scoring=scoring)
        results.append(cv_results)
        #print(f"results: {cv_results}")
        names.append(name)
        print(f"{name}: mean is {cv_results.mean()*100:.1f}% with SD {cv_results.std()*100.0:.1f}")
    # boxplot algorithm comparison
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.boxplot(results)
#    showmeans=True
    ax.set_xticklabels(names)
    plt.show()
            #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    
    
print("done")



In [ ]:
base_dir = '/Users/L/Downloads/palmarhyperlinearity/'
base_dir_thenar = '/Users/L/Downloads/palmarhyperlinearity/thenarversionall2/'
base_dir_p = '/Users/L/Downloads/palmarhyperlinearity/palmarversionall2/'

train_dir_flg_thenar = os.path.join(base_dir_thenar, 'flg')
train_dir_wt_thenar = os.path.join(base_dir_thenar, 'wtupdated')

train_dir_flg_palm = os.path.join(base_dir_p, 'flg')
train_dir_wt_palm = os.path.join(base_dir_p, 'wt')


print(f"Thenar WT photo count: {len(os.listdir(train_dir_wt_thenar))}")
print(f"Palmar WT photo count: {len(os.listdir(train_dir_wt_palm))}")

print(f"Thenar FLG photo count: {len(os.listdir(train_dir_flg_thenar))}")
print(f"Palmar FLG photo count: {len(os.listdir(train_dir_flg_palm ))}")

print(f"TOTAL photo count: {(len(os.listdir(train_dir_flg_palm))+len(os.listdir(train_dir_wt_palm)))}")

hetero = os.path.join(base_dir_p, 'het')
homo = os.path.join(base_dir_p, 'homo')
print(f"hrtro photo count: {len(os.listdir(hetero))}") ##I THINK THIS INCLUDES .FILE
print(f"homor photo count: {len(os.listdir(homo))}")
flg2 = os.path.join(base_dir_p, 'flg2')
print(f"joint photo count: {len(os.listdir(flg2))}") 


In [ ]:
##Joint settings
yy = 128  *4
xx = 64 * 4

###GLCM settings
distances = [1, 2, 4] 
angles = [0, np.pi/4, np.pi/2]
    #angles = [0, np.pi/4, np.pi/3]
symmetric=False
normed=True
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy',
                          'correlation', 'ASM']
###HOG settings
orientation=9
cellsperblock = (2, 2)
pixelspercell = (16, 16)



###lists
datahog = []
labelshog = []
dataglcm = []
labelsglcm = []
globalfeats = []
globallabels = []


###LISTS
datahog = []
labelshog = []
dataglcm = []
labelsglcm = []
globalfeats = []
globallabels = []

###FLG MUST BE PATH[0]
paths = [train_dir_flg_thenar, train_dir_wt_thenar]
imageloader(paths, filter_on=False, norm=False)

seed      = 9



In [ ]:
###GLCM TESTING 1 OF 2
print("[INFO] training classifier...")
models = []
models.append(('3NN', KNeighborsClassifier(n_neighbors=3)))
models.append(('5NN', KNeighborsClassifier(n_neighbors=5)))
models.append(('SVM-L', LinearSVC(random_state=seed, dual=False)))
models.append(('SVM-rbf', svm.SVC(kernel='rbf')))
print("done")

test_size=0.10
seed=9
kfoldsplits=10
datasetx = dataglcm
labelsx = labelsglcm
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                          random_state=seed                                                                           #shuffle=True) 
                                                                                                    
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))



In [ ]:
####GLCM 2 V2 thEnar

from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
specificity = make_scorer(recall_score, pos_label=0) #The label 0 is usually the negative class in a binary problem.
sensitivity = make_scorer(recall_score, pos_label=1)


results = []
names   = []

#scoring = 'accuracy'
#scoring = sensitivity
scoring = specificity


GLCMHOGtest2()                    



In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
"""
Analysis palms
"""
yy = 128 #*4
xx = 64 #* 4
###GLCMg
distances = [1, 2, 4] 
angles = [0, np.pi/4, np.pi/3]
symmetric=False
normed=True
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy',
                          'correlation', 'ASM']
###HOGh
orientation=9
cellsperblock = (2, 2)
pixelspercell = (16, 16)

datahog = []
labelshog = []
dataglcm = []
labelsglcm = []
globalfeats = []
globallabels = []

###FLG MUST BE PATH[0]
paths =
 [train_dir_flg_palm, train_dir_wt_palm]
imageloader(paths, filter_on=False, norm=False)

num_trees = 100
seed      = 9





In [ ]:
"""
GLCM Test 1/2 Palms
"""
print("[INFO] training classifier...")
models = []

models.append(('3NN', KNeighborsClassifier(n_neighbors=3)))
models.append(('5NN', KNeighborsClassifier(n_neighbors=5)))
models.append(('SVM-L', LinearSVC(random_state=seed, dual=False)))
models.append(('SVM-rbf', svm.SVC(kernel='rbf')))
print("done")

test_size=0.1
kfoldsplits=9
seed=9

datasetx = dataglcm
labelsx = labelsglcm
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)
                                                                                          #shuffle=True) 
                                                                                                    
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))



In [ ]:
"""
HOG Test 1/2 Palms
"""


datasetx = datahog
labelsx = labelshog
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                            random_state=seed)
                                                                                       #,                                                                   )
print("Split train and test")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))


In [ ]:
"""
GLCM Test 1/2 Palms
"""
print("[INFO] training classifier...")
models = []
#models.append(('LogReg', LogisticRegression(random_state=seed)))
models.append(('3NN', KNeighborsClassifier(n_neighbors=3)))
models.append(('5NN', KNeighborsClassifier(n_neighbors=5)))
models.append(('SVM-L', LinearSVC(random_state=seed, dual=False)))
models.append(('SVM-rbf', svm.SVC(kernel='rbf')))
print("done")

test_size=0.1
kfoldsplits=9
seed=9

datasetx = dataglcm
labelsx = labelsglcm
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)
                                                                                          #shuffle=True) 
                                                                                                    
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))


In [ ]:
"""
NEW GLCM
"""

In [ ]:
###GLCM TESTING 1 OF 2 thenar
print("[INFO] training classifier...")
models = []
models.append(('3NN', KNeighborsClassifier(n_neighbors=3)))
models.append(('5NN', KNeighborsClassifier(n_neighbors=5)))
models.append(('SVM-L', LinearSVC(random_state=seed, dual=False)))
models.append(('SVM-rbf', svm.SVC(kernel='rbf')))
print("done")

test_size=0.10
seed=9
kfoldsplits=10
datasetx = dataglcm
labelsx = labelsglcm
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)
                                                                                          #shuffle=True) 
                                                                                                    
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))





In [ ]:
###HOG TESTING 1 OF 2

datasetx = datahog
labelsx = labelshog
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                            random_state=seed)
                                                                                       #,                                                                   )
print("split train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))






In [ ]:
###HOG TESTING 2 OF 2
###GLCM 2 V2

from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
specificity = make_scorer(recall_score, pos_label=0) #The label 0 is usually the negative class in a binary problem.
sensitivity = make_scorer(recall_score, pos_label=1)


results = []
names   = []
#scoring = 'accuracy'
#scoring = sensitivity
scoring = specificity


GLCMHOGtest2()                    

   

"""
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
iris = load_iris()
scoring = ['precision', 'recall', 'f1']
clf = SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, iris.data, iris.target == 1, cv=5,
                        scoring=scoring, return_train_score=False)
"""

"""
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
"""

"""
 For each metric you can calculate mean and std value by using:
 np.mean(results[value]) 
 and 
 np.std(results[value])
 , where value - one of your specified metric name.
"""
                    



In [ ]:
"""
Analysis palms
"""
yy = 128 #*4
xx = 64 #* 4
###GLCMg
distances = [1, 2, 4] 
angles = [0, np.pi/4, np.pi/3]
symmetric=False
normed=True
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy',
                          'correlation', 'ASM']
###HOGh
orientation=9
cellsperblock = (2, 2)
pixelspercell = (16, 16)

datahog = []
labelshog = []
dataglcm = []
labelsglcm = []
globalfeats = []
globallabels = []

###FLG MUST BE PATH[0]
paths = [train_dir_flg_palm, train_dir_wt_palm]
imageloader(paths, filter_on=False, norm=False)

num_trees = 100
seed      = 9






In [ ]:
"""
GLCM Test 1/2 Palms
"""
print("[INFO] training classifier...")
models = []
models.append(('3NN', KNeighborsClassifier(n_neighbors=3)))
models.append(('5NN', KNeighborsClassifier(n_neighbors=5)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed)))
models.append(('SVM-L', LinearSVC(random_state=seed, dual=False)))
models.append(('SVM-rbf', svm.SVC(kernel='rbf')))
print("done")

test_size=0.1
kfoldsplits=9
seed=9

datasetx = dataglcm
labelsx = labelsglcm
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                          random_state=seed)
                                                                                          #shuffle=True) 
                                                                                                    
print("[STATUS] splitted train and test data...")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))



In [ ]:
                

   

"""
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
iris = load_iris()
scoring = ['precision', 'recall', 'f1']
clf = SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, iris.data, iris.target == 1, cv=5,
                        scoring=scoring, return_train_score=False)
"""

"""
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
"""

"""
 For each metric you can calculate mean and std value by using:
 np.mean(results[value]) 
 and 
 np.std(results[value])
 , where value - one of your specified metric name.
"""

##### """
HOG Test 1/2 Palms
"""
datasetx = datahog
labelsx = labelshog
trainDataGlobal, testDataGlobal, trainLabelsGlobal, testLabelsGlobal = train_test_split(np.array(datasetx),
                                                                                          np.array(labelsx),
                                                                                          test_size=test_size,
                                                                                            random_state=seed)          
print("Split train and test")
print("Train data  : {}".format(trainDataGlobal.shape))
print("Test data   : {}".format(testDataGlobal.shape))
print("Train labels: {}".format(trainLabelsGlobal.shape))
print("Test labels : {}".format(testLabelsGlobal.shape))



In [ ]:
###HOG PALMAR 2 V2

from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
specificity = make_scorer(recall_score, pos_label=0) #The label 0 is usually the negative class in a binary problem.
sensitivity = make_scorer(recall_score, pos_label=1)


results = []
names   = []

scoring = 'accuracy'
#scoring = sensitivity
#scoring = specificity


GLCMHOGtest2()                    


   

"""
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
iris = load_iris()
scoring = ['precision', 'recall', 'f1']
clf = SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, iris.data, iris.target == 1, cv=5,
                        scoring=scoring, return_train_score=False)
"""

"""
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
"""

"""
 For each metric you can calculate mean and std value by using:
 np.mean(results[value]) 
 and 
 np.std(results[value])
 , where value - one of your specified metric name.
"""